# Importer les bibliothèques nécessaires

In [2]:
import pandas as pd
import ast 
import numpy as np
import csv

# 

In [3]:
data = pd.read_csv('data/extracted_data_final_0_1000.csv')

In [4]:
data['Profile Cards'].iloc[8]

'{\'card_0\': "Carte et informations d\'accès\\n113VH\\n113 Avenue Victor Hugo, 75016 Paris\\nMoyens de transport\\nMétro - Victor Hugo (ligne 2)\\nBus - Place Jean Monnet (ligne 52)\\nBus - Bugeaud (ligne 52)\\nParking public\\nPlace Victor Hugo\\n80 Avenue Victor Hugo, Paris\\nInformations pratiques\\n1er étage avec ascenseur\\nParking payant\\nJ\'autorise le traitement d\'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus\\nsur la collecte et le traitement des données par Google\\nAFFICHER LA CARTE", \'card_1\': "Présentation de l\'établissement\\nTaddéo Alice vous reçoit dans son cabinet à Saint-Cloud.\\nLe chirurgien plastique et esthétique traite les conséquences morphologiques du vieillissement, de la grossesse ou de disgrâces acquises ou constitutionnelles non pathologiques. Il pratique notamment lifting cervico-facial, rhinoplastie, implants mammaires, lipoaspiration etc.", \'card_2\': "L\'équipe\\nSpécial

In [5]:
# Fonction pour réorganiser les profils cards
def reorder_cards(profile_cards):
    try:
        cards_dict = ast.literal_eval(profile_cards)
        
        reordered_cards = {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

        def split_content(value):
            value = value.replace('\\n', '\n').replace('\\', '\n')
            return value.split('\n')

        for key, value in cards_dict.items():
            #print(key,value,'s')
            if value.startswith("Carte et informations d'accès"):
                #print('1')
                reordered_cards['card_1'] = split_content(value)
            elif value.startswith("Présentation"):
                #print('2')
                reordered_cards['card_2'] = split_content(value)
            elif value.startswith("Horaires et coordonnées"):
                #print('3')
                reordered_cards['card_3'] = split_content(value)
            elif value.startswith("Tarifs et remboursement") or value.startswith("Lieu"):
                #print('i')
                reordered_cards['card_0'] = split_content(value)
            elif value.startswith("Tarifs"):
                #print('4')
                reordered_cards['card_4'] = split_content(value)
            elif value.startswith("Informations légales"):
                #print('5')
                reordered_cards['card_5'] = split_content(value)
            else :
                if isinstance(value,list):
                    #print('i1')
                    reordered_cards['card_0'] = split_content(value)
        
        return reordered_cards
    except:
        return {
            'card_0': np.nan,
            'card_1': np.nan,
            'card_2': np.nan,
            'card_3': np.nan,
            'card_4': np.nan,
            'card_5': np.nan
        }

reordered_cards = data['Profile Cards'].apply(reorder_cards)

reordered_cards_df = pd.json_normalize(reordered_cards)

data_combined = pd.concat([data, reordered_cards_df], axis=1)

data_combined.drop(columns = 'Profile Cards', inplace=True)
print(data_combined[['Doctor Name', 'card_0', 'card_1', 'card_2', 'card_3', 'card_4', 'card_5']].tail(9))

"""data2=data.iloc[998::]
reordered_cards = data2['Profile Cards'].apply(reorder_cards)
reordered_cards_df = pd.json_normalize(reordered_cards)
data_combined = pd.concat([data2, reordered_cards_df], axis=1)

data_combined.drop(columns = 'Profile Cards', inplace=True)
print(data_combined[['Doctor Name', 'card_0', 'card_1', 'card_2', 'card_3', 'card_4', 'card_5']].head(9))"""


                   Doctor Name  \
991        Dr Mathieu SABATIER   
992           Dr Eric LE FLOCH   
993    Agnès Montagnon -Thomas   
994     Dr Stella Danan-Hadida   
995  Dr Philippe Lazare SELLAM   
996          Dr Michèle Maizil   
997           Dr Thierry MEYER   
998          Dr Marc Stromboni   
999               Fabien GUIDE   

                                                card_0  \
991                                                NaN   
992  [Lieu 1, Lieu 2, Lieu 3, Radio Faidherbe, Radi...   
993  [Tarifs et remboursement, , , Tiers payant : S...   
994  [Tarifs et remboursement, , , , Conventionné s...   
995  [Lieu 1, Lieu 2, Cabinet du Docteur Philippe L...   
996                                                NaN   
997  [Tarifs et remboursement, , , , Conventionné, ...   
998  [Tarifs et remboursement, , Conventionné secte...   
999  [Tarifs et remboursement, , , , Conventionné, ...   

                                                card_1  \
991  [Carte et infor

"data2=data.iloc[998::]\nreordered_cards = data2['Profile Cards'].apply(reorder_cards)\nreordered_cards_df = pd.json_normalize(reordered_cards)\ndata_combined = pd.concat([data2, reordered_cards_df], axis=1)\n\ndata_combined.drop(columns = 'Profile Cards', inplace=True)\nprint(data_combined[['Doctor Name', 'card_0', 'card_1', 'card_2', 'card_3', 'card_4', 'card_5']].head(9))"

In [6]:
data['Profile Cards'].iloc[999]

'{\'card_0\': \'Tarifs et remboursement\\n\\n\\n\\nConventionné\\nCarte Vitale acceptée\\nTiers payant : Sécurité sociale\\nVoir les tarifs\\nMoyens de paiement\\nChèques, espèces et carte bancaire\\nExpertises et actes\\nBilan podologique\\nIonophorèse\\nK-Tape\\nKinesio Taping\\nOrthonyxie\\nOrthoplastie\\nOrthèse plantaire\\nPodologie\\nPédicurie\\nSemelles orthopédiques\\nVerrue\', \'card_1\': "Carte et informations d\'accès\\n9 Avenue Ampère, 77420 Champs-sur-Marne\\nMoyens de transport\\nBus - Ampere (lignes 213 et 312)\\nBus - Noisy-Champs Rer - Descartes (ligne 212)\\nInformations pratiques\\n1er étage avec ascenseur\\nEntrée accessible\\nJ\'autorise le traitement d\'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus\\nsur la collecte et le traitement des données par Google\\nAFFICHER LA CARTE", \'card_2\': "Présentation\\nDepuis janvier 2010, je vous reçois dans mon cabinet à Champs-sur-Marne à deux pas de

In [7]:
data_combined.head(9)

,Doctor Name,Specialty,Map Location,Skills,URL,card_0,card_1,card_2,card_3,card_4,card_5
0,Dr Marian AGACHI,Neurochirurgien,"{""lat"":48.8783278,""lng"":2.431033999999954,""ful...",Expertises et actes\nNeurochirurgie du rachis,https://www.doctolib.fr/neurochirurgien/paris/...,NaN,"[Carte et informations d'accès, Centre médico-...","[Présentation, Le docteur Marian Agachi vous a...","[Horaires et coordonnées, Horaires d'ouverture...",NaN,"[Informations légales, Numéro RPPS, 10001649291]"
1,Dr Anne Vaillant Moga,Médecin généraliste,"{""lat"":48.8692182,""lng"":2.285468,""fullAddress""...",Expertises et actes\nAcné\nAllergie\nApnée du ...,https://www.doctolib.fr/medecin-generaliste/pa...,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Carte et informations d'accès, Dr Anne Moga (...","[Présentation, Située au cœur du seizieme arr...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...","[Informations légales, Numéro RPPS, 1000037899..."
2,Dr Gabriel Ohana,Chirurgien-dentiste,"{""lat"":48.8941478,""lng"":2.2506978,""fullAddress...",Expertises et actes\nChirurgie buccale\nProthè...,https://www.doctolib.fr/dentiste/courbevoie/ga...,"[Tarifs et remboursement, , Carte Vitale accep...","[Carte et informations d'accès, Cabinet dentai...","[Présentation, Le docteur Gabriel Ohana vous a...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...","[Informations légales, Numéro RPPS, 10003639373]"
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"{""lat"":48.8154325,""lng"":2.3430171,""fullAddress...",Skills missing,https://www.doctolib.fr/dermatologue/gentilly/...,"[Tarifs et remboursement, , , Conventionné sec...","[Carte et informations d'accès, Villa Amélie, ...","[Présentation, Le docteur Thomas Bamberger vou...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...","[Informations légales, Numéro RPPS, 10003719571]"
4,Dr Layla Chatila,Chirurgien-dentiste,"{""lat"":48.8588045,""lng"":2.274392,""fullAddress""...",Skills missing,https://www.doctolib.fr/dentiste/paris/layla-c...,"[Tarifs et remboursement, , , , Conventionné, ...","[Carte et informations d'accès, 5 rue François...","[Présentation, Le docteur Layla Chatila vous a...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...","[Informations légales, Numéro RPPS, 10005133003]"
5,Dr David Karsenti,Gastro-entérologue et hépatologue,"{""lat"":48.8534353,""lng"":2.3326389,""fullAddress...",Expertises et actes\nGastro-entérologie\nProct...,https://www.doctolib.fr/gastro-enterologue/par...,"[Tarifs et remboursement, , Conventionné secte...","[Carte et informations d'accès, 50 rue de Renn...","[Présentation, Le docteur David Karsenti vous ...","[Horaires et coordonnées, Coordonnées, Numéro ...","[Tarifs, Consultation, 100 €, Ligature hémorro...","[Informations légales, Numéro RPPS, 1000153892..."
6,Jeannie NESTOR,Pédicure-podologue,"{""lat"":48.845705,""lng"":2.30262300000004,""fullA...",Expertises et actes\nBilan podologique\nPédicurie,https://www.doctolib.fr/pedicure-podologue/par...,NaN,"[Carte et informations d'accès, Centre de sant...","[Présentation, La pédicure-podologue Jeannie N...","[Horaires et coordonnées, Horaires d'ouverture...",NaN,"[Informations légales, Numéro RPPS, 10006076227]"
7,Dr Alice de Taddéo,Spécialiste en chirurgie plastique reconstruct...,"{""lat"":48.8685766,""lng"":2.282690600000024,""ful...",Expertises et actes\nChirurgie dermatologique,https://www.doctolib.fr/chirurgien-plastique/p...,NaN,"[Carte et informations d'accès, de Taddéo Alic...","[Présentation, Le docteur Alice De Taddéo, vou...",NaN,NaN,"[Informations légales, Numéro RPPS, 10100273183]"
8,de Taddéo Alice,Cabinet médical,"{""lat"":48.8685766,""lng"":2.282690600000024,""ful...",Skills missing,https://www.doctolib.fr/cabinet-medical/saint-...,NaN,"[Carte et informations d'accès, 113VH, 113 Ave...

In [8]:
#Pour rendre la localisation plus claire
def extract_lat_lng(map_location):
    try:
        location_dict = ast.literal_eval(map_location)
        return (location_dict['lat'], location_dict['lng'])
    except:
        return (None, None)

data_combined['Latitude'], data_combined['Longitude'] = zip(*data_combined['Map Location'].apply(extract_lat_lng))

data_combined.drop(columns = 'Map Location', inplace=True)


In [9]:
# Fonction pour traiter la colonne 'Skills'
def process_skills(skills):
    if skills == 'Skills missing':
        return np.nan
    else:
        return skills.split('\n')

data_combined['Skills Processed'] = data_combined['Skills'].apply(process_skills)

data_combined.drop(columns = 'Skills', inplace=True)

In [10]:
data_combined['Specialty'].value_counts()

Specialty
Chirurgien-dentiste                                                 152
Médecin généraliste                                                 104
Ophtalmologue                                                        75
Cabinet médical                                                      56
Spécialiste en chirurgie plastique reconstructrice et esthétique     40
                                                                   ... 
Chirurgien oral                                                       1
Acupuncteur                                                           1
Addictologue                                                          1
Chirurgien cancérologue                                               1
Diététicien                                                           1
Name: count, Length: 78, dtype: int64

In [11]:
data_combined['RPPS'] = data_combined['card_5'].apply(lambda x: x[2] if isinstance(x, list) and len(x) > 2 else np.nan)

data_combined.drop(columns = 'card_5', inplace = True)


In [12]:
data_combined['Nb Skills'] = data_combined['Skills Processed'].apply(lambda x: len(x)-1 if isinstance(x, list) else np.nan)

In [13]:
data_combined['Présentation']=''
data_combined['Langues parlées']=''
data_combined['Diplômes nationaux et universitaires']=''
data_combined['Autres formations']=''
data_combined['Expériences']=''
data_combined['Site web']=''
data_combined['Travaux et publications']=''
data_combined['Prix et distinctions']=''
data_combined['site']=0

liste_mots_clefs = ['Présentation','Langues parlées','Diplômes nationaux et universitaires','Formations','Autres formations','Expériences', 'Site web', 'Travaux et publications', 'Prix et distinctions']

for i in range(data_combined.shape[0]):
    
    if isinstance(data_combined['card_2'].iloc[i],list) : 
        liste_card2 = data_combined['card_2'].iloc[i][1:]
        sous_liste = ''
        label = liste_mots_clefs[0]
        for x in liste_card2 :
            if x in liste_mots_clefs :
                data_combined.loc[i, label] = sous_liste
                label = x
                sous_liste = ''
            elif x=='Voir le site':
                data_combined.loc[i, 'site'] = 1
            elif x=='▾ Voir plus':
                None
            else :
                sous_liste = sous_liste+(x)

In [14]:
data_combined['Diplômes nationaux et universitaires b']=data_combined['Diplômes nationaux et universitaires'].apply(lambda x: 1 if len(x)>0 else 0)

In [15]:
data_combined['Nb caractères présentation'] = data_combined['Présentation'].apply(lambda x: len(x))

In [16]:
data_combined['Autres formations b']=data_combined['Autres formations'].apply(lambda x: 1 if len(x)>0 else 0)

In [17]:
data_combined['Nb langues']=0

for i in range(data_combined.shape[0]):
    if isinstance(data_combined['Langues parlées'].iloc[i],str) : 
        liste_langue = (data_combined['Langues parlées'].iloc[i]).split()
        count = 0
        for x in liste_langue :
            
            if  x != 'et'  :
                count +=1
        data_combined.loc[i,'Nb langues']=count

In [18]:
data_combined['Expériences b']=data_combined['Expériences'].apply(lambda x: 1 if len(x)>0 else 0)

In [19]:
liste1 = ["Carte et informations d'accès",'Moyens de transport','Parking public','Informations pratiques',"J'autorise le traitement d'informations (dont mon adresse IP) et leur transfert hors UE par Google Maps (USA) afin d’afficher la carte.En savoir plus", 'sur la collecte et le traitement des données par Google','AFFICHER LA CARTE']
def carac1(card1):
    nb=0
    for phrase in card1:
        if  phrase not in liste1:
            nb += len(phrase)
    return nb
data_combined['Carac_card_1'] = data_combined['card_1'].apply(lambda x: carac1(x)if isinstance(x, list) else np.nan)

In [20]:
def transport(card1):
    if isinstance(card1, list):
        if 'Moyens de transport' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Transport'] = data_combined['card_1'].apply(transport)

In [21]:
def parking(card1):
    if isinstance(card1, list):
        if 'Parking public' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Parking'] = data_combined['card_1'].apply(parking)

In [22]:
def infos(card1):
    if isinstance(card1, list):
        if 'Informations pratiques' in card1:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Infos'] = data_combined['card_1'].apply(infos)

In [23]:
def coordonnées(card3):
    if isinstance(card3, list):
        if 'Coordonnées' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Coordonnées'] = data_combined['card_3'].apply(coordonnées) 

In [24]:
def sansRDV(card3):
    if isinstance(card3, list):
        if 'Consultations sans rendez-vous' in card3:
            return 1
        else:
            return 0
    else:
        return 0
data_combined['Sans RDV'] = data_combined['card_3'].apply(sansRDV)

In [25]:
data_combined.drop(columns=['URL','card_1','card_2'],inplace=True)
data_combined.head()

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,...,Nb caractères présentation,Autres formations b,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
0,Dr Marian AGACHI,Neurochirurgien,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1.0,...,748,0,3,0,154.0,1,0,1,1,0
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10.0,...,2538,1,2,1,180.0,1,0,1,0,0
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3.0,...,481,1,1,0,240.0,1,0,1,1,0
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,NaN,...,407,0,1,1,155.0,1,0,1,1,0
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,NaN,...,246,0,1,0,202.0,1,1,1,1,0


In [26]:
indices_nan = data_combined[data_combined['RPPS'].isna()].index.tolist()
print(indices_nan, len(indices_nan))

[8, 23, 27, 73, 74, 94, 112, 116, 117, 118, 121, 122, 123, 124, 127, 130, 131, 132, 135, 136, 138, 139, 140, 141, 142, 143, 145, 148, 150, 151, 152, 153, 156, 157, 158, 160, 161, 163, 166, 167, 169, 170, 172, 173, 175, 176, 177, 178, 179, 180, 182, 183, 184, 186, 187, 188, 190, 191, 192, 194, 196, 197, 198, 201, 202, 204, 209, 213, 214, 215, 216, 219, 220, 221, 222, 223, 225, 226, 227, 229, 230, 231, 233, 234, 235, 236, 238, 239, 240, 241, 245, 246, 247, 249, 250, 252, 254, 256, 259, 260, 262, 263, 264, 265, 267, 268, 270, 271, 272, 276, 277, 278, 279, 280, 281, 282, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 312, 315, 316, 317, 318, 319, 320, 321, 323, 324, 325, 326, 327, 328, 329, 331, 335, 337, 339, 344, 346, 347, 348, 349, 350, 351, 352, 353, 354, 356, 358, 359, 360, 365, 367, 368, 371, 372, 373, 374, 376, 378, 379, 380, 381, 382, 383, 384, 387, 388, 390, 392, 393, 395, 396, 397, 398, 399, 400, 4

In [27]:
data_combined_sans_nan = data_combined[data_combined['RPPS'].notna()]
data_combined_sans_nan

,Doctor Name,Specialty,card_0,card_3,card_4,Latitude,Longitude,Skills Processed,RPPS,Nb Skills,...,Nb caractères présentation,Autres formations b,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
0,Dr Marian AGACHI,Neurochirurgien,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.878328,2.431034,"[Expertises et actes, Neurochirurgie du rachis]",10001649291,1.0,...,748,0,3,0,154.0,1,0,1,1,0
1,Dr Anne Vaillant Moga,Médecin généraliste,"[Lieu 1, Lieu 2, Dr Anne Moga (Paris), 76 aven...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation préalable de médecine es...",48.869218,2.285468,"[Expertises et actes, Acné, Allergie, Apnée du...",10000378991,10.0,...,2538,1,2,1,180.0,1,0,1,0,0
2,Dr Gabriel Ohana,Chirurgien-dentiste,"[Tarifs et remboursement, , Carte Vitale accep...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation dentaire, 23 €, Traiteme...",48.894148,2.250698,"[Expertises et actes, Chirurgie buccale, Proth...",10003639373,3.0,...,481,1,1,0,240.0,1,0,1,1,0
3,Dr Thomas BAMBERGER,Dermatologue et vénérologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation, 31,50 € à 56,50 €, Cont...",48.815433,2.343017,NaN,10003719571,NaN,...,407,0,1,1,155.0,1,0,1,1,0
4,Dr Layla Chatila,Chirurgien-dentiste,"[Tarifs et remboursement, , , , Conventionné, ...","[Horaires et coordonnées, Coordonnées, 01 45 2...","[Tarifs, Consultation dentaire, 23 €, Ces hono...",48.858804,2.274392,NaN,10005133003,NaN,...,246,0,1,0,202.0,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
990,Dr Danielle Taconet,Ophtalmologue,"[Tarifs et remboursement, , , Conventionné sec...","[Horaires et coordonnées, Contact d'urgence, E...",NaN,49.060704,2.184674,"[Expertises et actes, Champ visuel, Laser opht...",10003749230,4.0,...,7800,0,2,0,231.0,1,0,1,1,0
994,Dr Stella Danan-Hadida,Gynécologue médicale et obstétrique,"[Tarifs et remboursement, , , , Conventionné s...","[Horaires et coordonnées, Horaires d'ouverture...","[Tarifs, Consultation vidéo, 58 €, Consultatio...",48.871909,2.346805,"[Expertises et actes, Gynécologie obstétrique,...",10000292291,7.0,...,626,0,1,0,249.0,1,1,1,1,0
995,Dr Philippe Lazare SELLAM,Spécialiste en chirurgie plastique reconstruct...,"[Lieu 1, Lieu 2, Cabinet du Docteur Philippe L...","[Horaires et coordonnées, Contact d'urgence, E...","[Tarifs, Consultation, 23 € à 120 €, Exérèse d...",48.868676,2.325944,NaN,10000558964,NaN,...,535,1,1,0,271.0,1,1,1,1,0
996,Dr Michèle Maizil,Chirurgien-dentiste,NaN,"[Horaires et coordonnées, Horaires d'ouverture...",NaN,48.834970,2.246637,NaN,10000202985,NaN,...,371,0,0,0,376.0,1,1,1,1,0


Ouverture annuaire santé

In [28]:
annuaire = pd.read_csv('data/PS_LibreAcces_Personne_activite_small.csv')
annuaire.columns

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_81949/1480950782.py:1: DtypeWarning: Columns (13,20) have mixed types. Specify dtype option on import or set low_memory=False.
  annuaire = pd.read_csv('data/PS_LibreAcces_Personne_activite_small.csv')


Index(['typedidentifiantpp', 'identifiantpp', 'codecivilitédexercice',
       'nomdexercice', 'prénomdexercice', 'codeprofession',
       'libelléprofession', 'codecatégorieprofessionnelle',
       'libellécatégorieprofessionnelle', 'codemodeexercice',
       'libellémodeexercice', 'complémentdestinatairecoordstruc',
       'complémentpointgéographiquecoord', 'numérovoiecoordstructure',
       'indicerépétitionvoiecoordstructu', 'codetypedevoiecoordstructure',
       'libellétypedevoiecoordstructure', 'libellévoiecoordstructure',
       'mentiondistributioncoordstructur', 'bureaucedexcoordstructure',
       'codepostalcoordstructure', 'codecommunecoordstructure',
       'libellécommunecoordstructure', 'codepayscoordstructure',
       'libellépayscoordstructure', 'codedépartementstructure',
       'libellédépartementstructure', 'codesecteurdactivité',
       'libellésecteurdactivité', 'coderôle', 'libellérôle',
       'codegenreactivité', 'libellégenreactivité'],
      dtype='object')

In [29]:
print(annuaire.shape[0],annuaire['codepostalcoordstructure'].isnull().sum())

2056801 582188


In [30]:
liste_med = annuaire.merge(data_combined,how='outer',left_on='identifiantpp',right_on='RPPS')

In [31]:
liste_med_2 = annuaire.merge(data_combined_sans_nan,how='outer',left_on='identifiantpp',right_on='RPPS')

In [32]:
print(liste_med_2.shape[0]-annuaire.shape[0], data_combined_sans_nan.shape[0],annuaire.shape[0])

3 367 2056801


In [33]:
liste_med_2_sans_nan = liste_med_2[liste_med_2['RPPS'].notna()]
print(liste_med_2_sans_nan.shape[0],liste_med_2_sans_nan['codepostalcoordstructure'].isnull().sum())

720 48


In [43]:
liste_med_2_sans_nan['RPPS'].value_counts()

RPPS
10004037247    9
10004402102    8
10002296464    8
10002292505    8
10005179634    7
              ..
10001625564    1
10001627800    1
10001645018    1
10002239217    1
950002402      1
Name: count, Length: 367, dtype: int64

In [49]:
regroupe = liste_med_2_sans_nan[liste_med_2_sans_nan['RPPS']=='10004037247']
regroupe.describe()

,typedidentifiantpp,codeprofession,codepayscoordstructure,codedépartementstructure,libellédépartementstructure,Latitude,Longitude,Nb Skills,site,Diplômes nationaux et universitaires b,Nb caractères présentation,Autres formations b,Nb langues,Expériences b,Carac_card_1,Transport,Parking,Infos,Coordonnées,Sans RDV
count,9.0,9.0,7.0,0.0,0.0,9.000000,9.000000,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0
mean,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,1.0,0.0,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0
std,0.0,0.0,0.0,NaN,NaN,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
min,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,1.0,0.0,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0
25%,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,1.0,0.0,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0
50%,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,1.0,0.0,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0
75%,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,1.0,0.0,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0
max,8.0,10.0,99000.0,NaN,NaN,50.648965,3.085347,10.0,1.0,0.0,623.0,0.0,0.0,0.0,274.0,1.0,1.0,1.0,1.0,0.0


In [52]:
df_sans_nan_dup = liste_med_2_sans_nan.drop_duplicates(subset=['RPPS'])
print(df_sans_nan_dup.shape[0],df_sans_nan_dup['codepostalcoordstructure'].isnull().sum())


367 34


In [59]:
df_sans_nan_dup_dep = df_sans_nan_dup[df_sans_nan_dup['codepostalcoordstructure'].notna()]

In [64]:
df_sans_nan_dup_dep['codemodeexercice'].value_counts()

codemodeexercice
L    275
S     58
Name: count, dtype: int64

In [63]:
df_sans_nan_dup_dep['Specialty'].value_counts()

Specialty
Chirurgien-dentiste                                                 66
Médecin généraliste                                                 37
Ophtalmologue                                                       29
Gynécologue obstétricienne                                          14
Pédicure-podologue                                                  14
Spécialiste en chirurgie plastique reconstructrice et esthétique    13
Dermatologue et vénérologue                                         13
Gynécologue obstétricien                                            12
Masseur-kinésithérapeute                                            12
Chirurgien orthopédiste et traumatologue                             9
Pédiatre                                                             8
Cardiologue                                                          8
Chirurgien urologue                                                  8
Gastro-entérologue et hépatologue                                  

In [42]:
print(liste_med.shape[0]-annuaire.shape[0], data_combined.shape[0],annuaire.shape[0])

636 1000 2056801 Index(['typedidentifiantpp', 'identifiantpp', 'codecivilitédexercice',
       'nomdexercice', 'prénomdexercice', 'codeprofession',
       'libelléprofession', 'codecatégorieprofessionnelle',
       'libellécatégorieprofessionnelle', 'codemodeexercice',
       'libellémodeexercice', 'complémentdestinatairecoordstruc',
       'complémentpointgéographiquecoord', 'numérovoiecoordstructure',
       'indicerépétitionvoiecoordstructu', 'codetypedevoiecoordstructure',
       'libellétypedevoiecoordstructure', 'libellévoiecoordstructure',
       'mentiondistributioncoordstructur', 'bureaucedexcoordstructure',
       'codepostalcoordstructure', 'codecommunecoordstructure',
       'libellécommunecoordstructure', 'codepayscoordstructure',
       'libellépayscoordstructure', 'codedépartementstructure',
       'libellédépartementstructure', 'codesecteurdactivité',
       'libellésecteurdactivité', 'coderôle', 'libellérôle',
       'codegenreactivité', 'libellégenreactivité', 'Doctor

In [61]:
liste_med_2_sans_nan.iloc[1]

typedidentifiantpp               8.0
identifiantpp            10000030766
codecivilitédexercice             DR
nomdexercice                  NALLET
prénomdexercice             EMMANUEL
                            ...     
Transport                        0.0
Parking                          0.0
Infos                            0.0
Coordonnées                      1.0
Sans RDV                         0.0
Name: 7971, Length: 64, dtype: object

In [45]:
df_sans_nan = liste_med[liste_med['RPPS'].notna()]
df_sans_nan['codedépartementstructure'].isnull().sum()

720

Ouverture BAN

In [34]:
ban = pd.read_csv('data/adresses-france.csv')
ban.head()

/var/folders/rf/dqdwsk6s06ngm9b4j17fkb2r0000gn/T/ipykernel_81620/259408599.py:1: DtypeWarning: Columns (5,8,9,15,16,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  ban = pd.read_csv('data/adresses-france.csv')


,id,id_fantoir,numero,rep,nom_voie,code_postal,code_insee,nom_commune,code_insee_ancienne_commune,nom_ancienne_commune,...,type_position,alias,nom_ld,libelle_acheminement,nom_afnor,source_position,source_nom_voie,certification_commune,cad_parcelles,Unnamed: 23
0,01001_4b50r5_00630,NaN,630,NaN,la Chèvre,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,segment,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,LA CHEVRE,inconnue,inconnue,0,NaN,NaN
1,01001_g0ru02_00108,NaN,108,NaN,Clemencia,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,parcelle,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,CLEMENCIA,cadastre,cadastre,0,01001000ZL0263,NaN
2,01001_ngzlqw_00009,NaN,9,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN,NaN
3,01001_ngzlqw_00023,NaN,23,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN,NaN
4,01001_ngzlqw_00026,NaN,26,NaN,Imp des Epis,1400,1001,L'Abergement-Clémenciat,NaN,NaN,...,NaN,NaN,NaN,L'ABERGEMENT-CLEMENCIAT,IMP DES EPIS,arcep,arcep,0,NaN,NaN
